<a href="https://colab.research.google.com/github/hyunee9/AI/blob/main/%EC%8B%AC%EC%B8%B5%ED%95%99%EC%8A%B5_%EC%83%9D%EC%84%B1%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 라이브러리

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim #gradient descent method를 이용할 것이니
import torchvision # 영상 시각 지능에 특화된 라이브러리
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid #결과 출력할 때 5x5 영상 크기로 만들려고
from torch.utils.data import DataLoader #우리가 원하는 형식으로 데이터를 로딩할 수 있도록 도와주는
import matplotlib.pyplot as plt

# 2. model 정의

## 2.1 변수 설정

In [ ]:
latent_size = 62 #GAN의 입력으로 사용하는 데이터의 latent벡터의 크기
hidden_size = 256 #GAN의 FC Layer, hidden layer에 속하는 노드의 갯수
image_size = 28*28 #입력으로 사용하는 영상의 크기
batch_size = 64

## 2.2 generator

In [ ]:
#generator도 nn모델이기 때문에 nn.module을 상속받아서 만든다.
class Generator(nn.Module):
  def __init__(self):
    super(Generator, self).__init__()
    #fully connected layer이기 때문에 linear layer만 사용한다
    # 3개의 fully  connected layer를 사용 -> ReLU, ReLU, Tanh
    self.fc = nn.Sequential(
        #1st FC : latenet_size -> hidden_size
        nn.Linear(latent_size, hidden_size),
        nn.ReLU(),
        #2nd FC ; hidden_size -> hidden_size
        #Linear : hidden을 받아서 hidden만큼 출력을 해준다
        nn.Linear(hidden_size, hidden_size),
        nn.ReLU(),
        #3rd FC : hidden_size -> image_image
        nn.Linear(hidden_size, image_size),
        nn.Tanh()
    )

  def forward(self,x):
    return self.fc(x)

## 2.3 discriminator

In [ ]:
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator, self).__init__()
    self.fc = nn.Sequential(
        # 1st FC : image_size -> hidden_size / ReLU 사용
        nn.Linear(image_size, hidden_size),
        nn.ReLU(),

        # 2nd FC : hidden_size -> hidden_size / ReLU 사용
        nn.Linear(hidden_size, hidden_size),
        nn.ReLU(),

        # 3rd FC : hidden_size -> 1 / hidden_size로 부터 값을 뽑아내는데, 0~1 사이의 값이여야함.
        #그래서 sigmoid함수를 이용한다.
        nn.Linear(hidden_size, 1),
        nn.Sigmoid()
    )

  def forward(self,x):
    return self.fc(x)

# 3. data loading

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean = [0.5], std = [0.5])])

mnist = MNIST (root ='data/', train=True, transform=transform, download = True)

#데이터를 훈련시키기 위해서 정해진 배치 사이즈만큼 잘라서 넣는 것.
data_loader = DataLoader (dataset = mnist,
                          batch_size = batch_size,
                          shuffle = True,
                          num_workers = 2,
                          drop_last = True)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 35990241.68it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1091696.13it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 8726346.51it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 6133460.65it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



# 4. 초기 설정

In [ ]:
n_epochs = 100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

G = Generator().to(device)
D = Discriminator().to(device)

#loss 함수
criterion = nn.BCELoss()
d_optimizer = optim.Adam(D.parameters(), lr = 0.0002, betas = (0.5, 0.999))
g_optimizer = optim.Adam(G.parameters(),lr = 0.0002, betas = (0.5, 0.999))

## 4.1 결과 출력 함수 : image를(5x5)로 출력

In [ ]:
def show_images(images, num_images = 25, size = (1, 28, 28)):
  print(images.shape)
  image_flat = images.detach().cpu().view(-1, *size)
  image_grid = make_grid(image_flat[:num_images], nrow=5)
  plt.imshow(image_grid.permute(1,2,0).squeeze())
  plt.show()

# 5. 훈련

# 6. 실행

In [ ]:
# n_epochs 만큼 수행
for epoch in range(n_epochs):
  #data_loader에서 batch_size만큼 data 읽어오기
  for i, (images, _) in enumerate(data_loader):
    #1. flatten the images
    images = images.reshape(batch_size, -1).to(device)
    #to(device) : device로 보내서 GPU상에서 계산해달라는 뜻

    #2. label 설정 : real -> 1, fake -> 0
    real_labels = torch.ones(batch_size, 1).to(device)
    fake_labels = torch.zeros(batch_size, 1).to(device)

    #3. Discriminator 훈련

    # 3.1 real image에 대한 loss 계산
    #discriminator에 real image를 넣어서 결과를 출력
    output = D(images)
    #output에 대한 loss를 계산 : loss : y_hat와 y의 차이
    # -> y_hat : output, y : real_labels
    d_loss_real = criterion(output, real_labels)



    #3.2 fkae image에 대한 loss 계산
    #latent vector부터 생성
    z = torch.randn(batch_size, latent_size).to(device)
    #fake image 생성
    fake_images = G(z)
    #discriminator에 fake image를 넣어서 결과를 출력
    output = D(fake_images.detach())
    #.detach () : 더이상 미분하지 않겠다.
    #loss 함수 구하기
    d_loss_fake = criterion(output, fake_labels)

    d_loss = d_loss_real + d_loss_fake

    #3.3 gradient -> gradient descent
    d_optimizer.zero_grad()
    d_loss.backward()
    d_optimizer.step()

    #4. Generator를 훈련
    #4.1 fake image를 생성해서 loss 계산
    z = torch.randn(batch_size, latent_size).to(device)
    #fake image 생성
    fake_images = G(z)
    #discriminator에 fake image를 넣어서 결과를 출력
    output = D(fake_images)
    g_loss = criterion(output, real_labels)

    #4.2 gradient -> gradient descent
    g_optimizer.zero_grad()
    g_loss.backward()
    g_optimizer.step()


    show_images(fake_images) # 가짜 이미지
    show_images(images) #실제 이미지


NameError: name 'n_epochs' is not defined

In [ ]:
z = torch.randn(batch_size, latent_size).to(device)
fake_image = G(z)